###### In this assignment students need to predict whether a person makes over 50K per year or not from classic adult dataset using XGBoost.
###### trainig DataSet- http://archive.ics.uci.edu/ml/machine-learningdatabases/adult/adult.data
###### test DataSet- http://archive.ics.uci.edu/ml/machine-learningdatabases/adult/adult.test

In [35]:
import numpy as np
import pandas as pd
import xgboost as xgb

In [196]:
train_set = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header = None)
test_set = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test',skiprows = 1, header = None)

In [197]:
print("Train_shape :",train_set.shape)
print("Test_shape :",test_set.shape)

Train_shape : (32561, 15)
Test_shape : (16281, 15)


In [198]:
train_set.rename(columns={0:'age',1:'workclass',2:'fnlwgt',3:'education',4:'education_num',5:'marital_status',6:'occupation',7:'relationship',8:'race',9:'sex',10:'capital_gain',11:'capital_loss',12:'hours_per_week',13:'native_country',14:'wage_class'},inplace=True)
train_set.head(2)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K


In [199]:
test_set.rename(columns={0:'age',1:'workclass',2:'fnlwgt',3:'education',4:'education_num',5:'marital_status',6:'occupation',7:'relationship',8:'race',9:'sex',10:'capital_gain',11:'capital_loss',12:'hours_per_week',13:'native_country',14:'wage_class'},inplace=True)
test_set.head(2)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.


In [200]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
wage_class        32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [201]:
train_set['wage_class'] = train_set['wage_class'].map({' <=50K':'<=50K', ' >50K': '>50K'})

In [202]:
#drop unidentified value from training dataset
train_set.replace(' ?', np.nan).dropna().shape

(30162, 15)

In [203]:
#drop unidentified value from test dataset
test_set.replace(' ?', np.nan).dropna().shape

(15060, 15)

In [204]:
set(train_set.workclass)

{' ?',
 ' Federal-gov',
 ' Local-gov',
 ' Never-worked',
 ' Private',
 ' Self-emp-inc',
 ' Self-emp-not-inc',
 ' State-gov',
 ' Without-pay'}

In [205]:
#work class train data into categorical clolumn
wrkcls_mask=pd.get_dummies(train_set['workclass'],prefix='wrk').iloc[:,1:]
train= pd.concat([train_set,wrkcls_mask],axis=1)
del train['workclass']

#work class test data into categorical clolumn
wrkcls_test_mask=pd.get_dummies(test_set['workclass'],prefix='wrk').iloc[:,1:]
test= pd.concat([test_set,wrkcls_test_mask],axis=1)
del test['workclass']

In [206]:
#Educaion class train data into categorical clolumn
edu_mask=pd.get_dummies(train['education'],prefix='edu').iloc[:,1:]
train=pd.concat([train,edu_mask],axis=1)
del train['education']

#Education class test data into categorical clolumn
edu_test_mask=pd.get_dummies(test['education'],prefix='edu').iloc[:,1:]
test=pd.concat([test,edu_test_mask],axis=1)
del test['education']

In [207]:
#marital status class train data into categorical clolumn
ms_mask=pd.get_dummies(train['marital_status'],prefix='mar_stat').iloc[:,1:]
train=pd.concat([train,ms_mask],axis=1)
del train['marital_status']

#marital status class test data into categorical clolumn
ms_test_mask=pd.get_dummies(test['marital_status'],prefix='mar_stat').iloc[:,1:]
test=pd.concat([test,ms_test_mask],axis=1)
del test['marital_status']

In [208]:
#Occupation class train data into categorical clolumn
ocu_mask=pd.get_dummies(train['occupation'],prefix='occu').iloc[:,1:]
train=pd.concat([train,ocu_mask],axis=1)
del train['occupation']

#Occupation class test data into categorical clolumn
ocu_test_mask=pd.get_dummies(test['occupation'],prefix='occu').iloc[:,1:]
test=pd.concat([test,ocu_test_mask],axis=1)
del test['occupation']

In [209]:
#Relationship class train data into categorical clolumn
rel_mask=pd.get_dummies(train['relationship'],prefix='reln').iloc[:,1:]
train=pd.concat([train,rel_mask],axis=1)
del train['relationship']

#Relationship class test data into categorical clolumn
rel_test_mask=pd.get_dummies(test['relationship'],prefix='reln').iloc[:,1:]
test=pd.concat([test,rel_test_mask],axis=1)
del test['relationship']

In [210]:
#Race class train data into categorical clolumn
race_mask=pd.get_dummies(train['race'],prefix='race').iloc[:,1:]
train=pd.concat([train,race_mask],axis=1)
del train['race']

#Race class test data into categorical clolumn
race_test_mask=pd.get_dummies(test['race'],prefix='race').iloc[:,1:]
test=pd.concat([test,race_test_mask],axis=1)
del test['race']

In [211]:
#Sex class train data into categorical clolumn
sex_mask=pd.get_dummies(train['sex'],prefix='sex').iloc[:,1:]
train=pd.concat([train,sex_mask],axis=1)
del train['sex']

#Sex class test data into categorical clolumn
sex_test_mask=pd.get_dummies(test['sex'],prefix='sex').iloc[:,1:]
test=pd.concat([test,sex_test_mask],axis=1)
del test['sex']

In [212]:
#Native country class test data into categorical clolumn
nc_mask=pd.get_dummies(train['native_country'],prefix='country').iloc[:,1:]
train=pd.concat([train,nc_mask],axis=1)
del train['native_country']

#Native country class test data into categorical clolumn
nc_test_mask=pd.get_dummies(test['native_country'],prefix='country').iloc[:,1:]
test=pd.concat([test,nc_test_mask],axis=1)
del test['native_country']

In [213]:
#Wages class test data into categorical clolumn
wag_mask=pd.get_dummies(train['wage_class'],prefix='wage').iloc[:,1:]
train=pd.concat([train,wag_mask],axis=1)
del train['wage_class']

#Wages class test data into categorical clolumn
wag_test_mask=pd.get_dummies(test['wage_class'],prefix='wage').iloc[:,1:]
test=pd.concat([test,wag_test_mask],axis=1)
del test['wage_class']

In [214]:
train.head(2)

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,wrk_ Federal-gov,wrk_ Local-gov,wrk_ Never-worked,wrk_ Private,...,country_ Puerto-Rico,country_ Scotland,country_ South,country_ Taiwan,country_ Thailand,country_ Trinadad&Tobago,country_ United-States,country_ Vietnam,country_ Yugoslavia,wage_>50K
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [215]:
test.head(2)

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,wrk_ Federal-gov,wrk_ Local-gov,wrk_ Never-worked,wrk_ Private,...,country_ Puerto-Rico,country_ Scotland,country_ South,country_ Taiwan,country_ Thailand,country_ Trinadad&Tobago,country_ United-States,country_ Vietnam,country_ Yugoslavia,wage_ >50K.
0,25,226802,7,0,0,40,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1,38,89814,9,0,0,50,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [216]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Columns: 101 entries, age to wage_>50K
dtypes: int64(6), uint8(95)
memory usage: 4.4 MB


In [217]:
dtrain = xgb.DMatrix(train)
dtest = xgb.DMatrix(test)

In [219]:
print("Train dataset contains {0} rows and {1} columns".format(train.num_row(), train.num_col()))
print("Test dataset contains {0} rows and {1} columns".format(test.num_row(), test.num_col()))

Train dataset contains 32561 rows and 101 columns
Test dataset contains 16281 rows and 100 columns


In [220]:
print("Train possible labels: ")
print(np.unique(train.get_label()))

print("\nTest possible labels: ")
print(np.unique(test.get_label()))

Train possible labels: 
[]

Test possible labels: 
[]


In [224]:
params = {
    'objective':'binary:logistic',
    'max_depth':2,
    'silent':1,
    'eta':1
}

num_rounds = 5

In [225]:
bst = xgb.train(params, train, num_rounds)

XGBoostError: b'[10:09:55] src/objective/regression_obj.cu:65: Check failed: info.labels_.Size() != 0U (0 vs. 0) label set cannot be empty\n\nStack trace returned 7 entries:\n[bt] (0) 0   libxgboost.dylib                    0x0000000116147120 dmlc::StackTrace() + 288\n[bt] (1) 1   libxgboost.dylib                    0x0000000116146ebf dmlc::LogMessageFatal::~LogMessageFatal() + 47\n[bt] (2) 2   libxgboost.dylib                    0x00000001161bd889 xgboost::obj::RegLossObj<xgboost::obj::LinearSquareLoss>::GetGradient(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo const&, int, xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*) + 265\n[bt] (3) 3   libxgboost.dylib                    0x0000000116143239 xgboost::LearnerImpl::UpdateOneIter(int, xgboost::DMatrix*) + 1017\n[bt] (4) 4   libxgboost.dylib                    0x000000011615ec24 XGBoosterUpdateOneIter + 180\n[bt] (5) 5   libffi.6.dylib                      0x0000000110fbd884 ffi_call_unix64 + 76\